<a href="https://colab.research.google.com/github/SantanaC4/pos_tagging_treebank/blob/main/pos_tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criação de um Part of Speech Tagging com base no Treebank corpus



In [ ]:
import pandas as pd

In [34]:
##Upload Penn Treebank Corpus
# That function filter the pair (word, tag) of the data
def pair_filter(section):
  if (section == "training"):
    url = 'https://raw.githubusercontent.com/SantanaC4/pos_tagging_treebank/main/Sec0-18_training'
  if (section == "development"):
    url = 'https://raw.githubusercontent.com/SantanaC4/pos_tagging_treebank/main/Sec-19-21_development'
  df = pd.read_csv(url, header=None, sep='\n')
  spliting =  [i.split(" ") for i in df[0]]
  extracting_pair = []
  for i in spliting:
    if (len(i) == 1):
      extracting_pair.append(tuple(i[0].split("_")))
    else:
      for j in i:
        if (j != ''):
          extracting_pair.append(tuple(j.split("_")))
  return (extracting_pair)

##Unigram Tagger 
####Building a dictionary with tags probability to each word


In [36]:
## Default dictionary 
## Assign default value to key that is not in dictionary
from collections import defaultdict

pos = defaultdict(list)
for (word, tag) in pair_filter("training"):
  pos[word].append(tag)

aux = defaultdict(int)
tags_probability = {}
word_freq = 0

for (word, tags) in pos.items():

  word_freq = len(tags)
  for i in tags:
    aux[i] += 1
  tags_probability[word] = sorted([(value/word_freq, key) for (key, value) in aux.items()], reverse=True)

  aux = defaultdict(int)
  word_freq = 0

print(tags_probability['is'])

[(0.9997030438010394, 'VBZ'), (0.0002969561989606533, 'NNS')]


In [43]:
tt = pair_filter("development")
tt[0]
for (word, tag) in tt:
  tags_probability[]

('The', 'DT')